### Setup

1. Make a copy of `../.env.example` to `../.env` and fill the `OPENAI_API_KEY` and `CHAINLIT_API_KEY` environment variables.
2. This demo assumes that chainlit-cloud is running locally.
3. Run the cells to install the dependencies and the local sdk package.

In [ ]:
pip install -r ../requirements.txt

In [ ]:
pip install -e ..

### Example

In [ ]:
import asyncio
import json

from chainlit_client import ChainlitClient
from chainlit_client.types import Attachment
from dotenv import load_dotenv
from openai import OpenAI
from requests import get

load_dotenv()

client = OpenAI()

sdk = ChainlitClient(batch_size=2)
sdk.instrument_openai()

thread_id = None

In [ ]:
@sdk.step(type="RUN")
def get_completion(welcome_message, text, temperature):
    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system",
                "content": f"Tell an inspiring quote to the user, mentioning their name. Be extremely supportive while keeping it short. Write one sentence per line. The current temperature is {temperature}°C.",
            },
            {
                "role": "assistant",
                "content": welcome_message,
            },
            {
                "role": "user",
                "content": text,
            },
        ],
    )
    return completion.choices[0].message.content

In [ ]:

current_weather_paris = "https://api.open-meteo.com/v1/meteofrance?latitude=48.8588475&longitude=2.3058359&hourly=temperature_2m,relative_humidity_2m&forecast_days=1"

@sdk.thread
def run():
    global thread_id
    thread_id = sdk.get_current_thread_id()

    welcome_message = "What's your name? "
    sdk.event(message=welcome_message, role="SYSTEM")
    text = input(welcome_message)
    sdk.event(message=text, role="USER")

    with sdk.step(type="TOOL") as step:
        import datetime
        res = get(current_weather_paris)
        json = res.json()

        current_time = datetime.datetime.now().strftime("%Y-%m-%dT%H:00")
        index = json["hourly"]["time"].index(current_time)
        temperature = json['hourly']['temperature_2m'][index]

        completion = get_completion(welcome_message=welcome_message, text=text, temperature=temperature)

    print("")
    print(completion)
    sdk.event(message=completion, role="ASSISTANT")
        

run()
sdk.wait_until_queue_empty()

In [ ]:
# Use the API to count the conversations in this project
# and add feedback to the LLM step of the current conversation

async def main():
    threads = await sdk.api.list_threads()
    print(threads["data"]["threads"]["totalCount"], "threads")

    print("\nSearching for the thread", thread_id, "...")
    thread = await sdk.api.get_thread(id=thread_id)

    print(json.dumps(thread.to_dict(), indent=2))

    # get the LLM step
    llm_step = [step for step in thread.steps if step.type == "LLM"][0]

    if not llm_step:
        print("Error: No LLM step found")
        return

    # attach a feedback
    await sdk.api.set_human_feedback(
        thread_id=thread_id, step_id=llm_step.id, value=1, comment="this was accurate"
    )

    # get the updated steps
    thread = await sdk.api.get_thread(id=thread_id)

    print(
        json.dumps(
            [step.to_dict() for step in thread.steps if step.type == "LLM"],
            indent=2,
        )
    )


asyncio.run(main())